In [3]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
pd.set_option('display.float_format', '{:.2f}'.format)

In [4]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_Movies_and_TV.json.gz

--2020-04-20 10:45:24--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_Movies_and_TV.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 281498988 (268M) [application/octet-stream]
Saving to: ‘meta_Movies_and_TV.json.gz’

meta_Movies_and_TV. 100%[===================>] 268.46M  11.0MB/s    in 25s     

2020-04-20 10:45:49 (10.9 MB/s) - ‘meta_Movies_and_TV.json.gz’ saved [281498988/281498988]



In [5]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Movies_and_TV.json.gz

--2020-04-20 10:45:50--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Movies_and_TV.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1538847239 (1.4G) [application/octet-stream]
Saving to: ‘Movies_and_TV.json.gz’

Movies_and_TV.json. 100%[===================>]   1.43G  11.0MB/s    in 2m 35s  

2020-04-20 10:48:26 (9.47 MB/s) - ‘Movies_and_TV.json.gz’ saved [1538847239/1538847239]



In [23]:
# open the json.gz file for meta dataset
data_meta = []
with gzip.open('meta_Movies_and_TV.json.gz') as f:
    for l in f:
        data_meta.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(data_meta))

# first row of the list
print(data_meta[0])

203970
{'category': ['Movies & TV', 'Movies'], 'title': 'Understanding Seizures and Epilepsy', 'rank': '886,503inMoviesTV(', 'main_cat': 'Movies & TV', 'asin': '0000695009'}


In [33]:
# convert data_meta to df
df_meta = pd.DataFrame.from_dict(data_meta)

# drop useless columns
df_meta_1 = df_meta.dropna(subset=['details'])
df_meta_1 = df_meta_1.drop(['image', 'feature', 'date', 'tech1'], axis=1)

In [35]:
# df_meta_1 = df_meta.dropna(subset=['details'])
df_meta_1.head()

,category,title,rank,main_cat,asin,description,brand,also_buy,also_view,price,details
8374,"[Movies & TV, Art House & International, By Co...",Olivier Olivier VHS,"343,783inMoviesTV(",Movies & TV,6303023045,"[Former rental,Subtitled The box has stickers ...",Frdric Quiring,NaN,NaN,NaN,"\n <div class=""content"">\n\n\n\n\n\n\n<ul..."
8375,"[Movies & TV, Genre for Featured Categories, D...",Georgia VHS,"321,811inMoviesTV(",Movies & TV,6303032559,[Drama],Judy Davis,NaN,NaN,$1.99,"\n <div class=""content"">\n\n\n\n\n\n\n<ul..."
8376,"[Movies & TV, Studio Specials, Universal Studi...",Rhythm Country & Blues VHS,"234,584inMoviesTV(",Movies & TV,630303831X,NaN,Gladys Knight,NaN,NaN,$38.99,"\n <div class=""content"">\n\n\n\n\n\n\n<ul..."
8377,"[Movies & TV, Genre for Featured Categories, C...",Good Morning Vietnam VHS,"200,800inMoviesTV(",Movies & TV,6303027016,[Academy Award(R)-winner Robin Williams shakes...,Robin Williams,"[6301627768, B000005PFL, 0792116887, 079070125...",NaN,$7.99,"\n <div class=""content"">\n\n\n\n\n\n\n<ul..."
8378,"[Movies & TV, Genre for Featured Categories, E...",Fitness Pros: Step Aerobics VHS,"155,699inMoviesTV(",Movies & TV,6303036449,"[ESPN's ""Fitness Pros"" take the viewer through...",NaN,[6303036422],NaN,$4.49,"\n <div class=""content"">\n\n\n\n\n\n\n<ul..."


In [ ]:
# opent the json.gz file for movies and TV reviews dataset
data_movie = []
with gzip.open('Movies_and_TV.json.gz') as f:
    for l in f:
        data_movie.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(data_movie))

# first row of the list
print(data_movie[0])

In [11]:
# clean the movie dataset by only selecting last three years and verified reviews, drop coulmn 'image'.
df_movie = pd.DataFrame.from_dict(data_movie)

df_movie_1 = df_movie[df_movie['reviewTime'].str.contains('2016') 
| df_movie['reviewTime'].str.contains('2017') 
| df_movie['reviewTime'].str.contains('2018')]

df_movie_1 = df_movie_1.drop('image', axis=1)
df_movie_1 = df_movie_1[df_movie_1['verified']==True]

In [36]:
# merge movie dataset and meta dataset
df_movie_meta = df_movie_1.merge(df_meta_1, on='asin', how='inner')

In [13]:
# pickle the df file for future use

# import pickle
# pickle.dump(df_movie_meta, open("df_movie_meta.pkl", "wb"))

In [16]:
# data_movies = pd.read_pickle('df_movie_meta.pkl')

In [37]:
#downsize the data to 2018
df_movie_2018 = df_movie_meta[df_movie_meta['reviewTime'].str.contains('2018')]

In [38]:
len(df_movie_2018)

209060

In [39]:
df_movie_2018.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,...,category,title,rank,main_cat,description,brand,also_buy,also_view,price,details
0,5.00,True,"04 8, 2018",A1CW3NLH9MBQRY,6303022901,{'Format:': ' DVD'},Sally Nunez,I really enjoyed this movie. Brings tears to m...,Five Stars,1523145600,...,"[Movies & TV, Genre for Featured Categories, D...",The Joy Luck Club VHS,"295,699inMoviesTV(",Movies & TV,[Produced by Academy Award(R)-winning filmmake...,Tamlyn Tomita,NaN,NaN,$3.28,"\n <div class=""content"">\n\n\n\n\n\n\n<ul..."
1,5.00,True,"04 2, 2018",A1WK0IRZ08NX9X,6303022901,{'Format:': ' DVD'},Ammie28,"If you have never seen this movie, you may jus...",Heart Touching...,1522627200,...,"[Movies & TV, Genre for Featured Categories, D...",The Joy Luck Club VHS,"295,699inMoviesTV(",Movies & TV,[Produced by Academy Award(R)-winning filmmake...,Tamlyn Tomita,NaN,NaN,$3.28,"\n <div class=""content"">\n\n\n\n\n\n\n<ul..."
2,5.00,True,"03 30, 2018",A2HGXJQCQTXE4E,6303022901,{'Format:': ' Blu-ray'},Pa nhia lee,awesome all time fave movie.,Five Stars,1522368000,...,"[Movies & TV, Genre for Featured Categories, D...",The Joy Luck Club VHS,"295,699inMoviesTV(",Movies & TV,[Produced by Academy Award(R)-winning filmmake...,Tamlyn Tomita,NaN,NaN,$3.28,"\n <div class=""content"">\n\n\n\n\n\n\n<ul..."
3,5.00,True,"03 30, 2018",A14ASTA78EK120,6303022901,{'Format:': ' DVD'},lalush,The greatest movie ever!!!!!,Five Stars,1522368000,...,"[Movies & TV, Genre for Featured Categories, D...",The Joy Luck Club VHS,"295,699inMoviesTV(",Movies & TV,[Produced by Academy Award(R)-winning filmmake...,Tamlyn Tomita,NaN,NaN,$3.28,"\n <div class=""content"">\n\n\n\n\n\n\n<ul..."
4,5.00,True,"03 29, 2018",A2EGT1RXKVOXTJ,6303022901,{'Format:': ' Amazon Video'},Karin,Love this movie,Five Stars,1522281600,...,"[Movies & TV, Genre for Featured Categories, D...",The Joy Luck Club VHS,"295,699inMoviesTV(",Movies & TV,[Produced by Academy Award(R)-winning filmmake...,Tamlyn Tomita,NaN,NaN,$3.28,"\n <div class=""content"">\n\n\n\n\n\n\n<ul..."


In [40]:
df_movie_2018.to_csv('movie_2018.csv', index=False)

In [49]:
from bs4 import BeautifulSoup

In [132]:
movie_2018 = pd.read_csv('movie_2018.csv')

In [134]:
links = []
for i in movie_2018['details']:
    soup = BeautifulSoup(i)
    found_links = soup.select('a.a-text-normal')
    if found_links:
        link = found_links[0]['href']
        links.append(link)
    else:
        links.append("")

In [ ]:
movie_2018['links'] = links
movie_2018.to_csv('2018_movie_links.csv')

In [130]:
df_links = pd.DataFrame(links, columns=['links'])